## Preparação do ambiente

In [1]:
! pip install pandas 
! pip install numpy 
! pip install plotly 
! pip install warnings 

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement warnings (from versions: none)
ERROR: No matching distribution found for warnings


In [1]:
# Importando bibliotecas necessárias
import pandas as pd 
import numpy as np 

import plotly.express as px 
import plotly.graph_objects as go 
from plotly.subplots import make_subplots

import warnings 
warnings.filterwarnings('ignore')

## Tratamento inicial 

In [2]:
# Lendo o dataframe 
base_eletricidade = pd.read_csv('electricity_prices.csv')

In [3]:
base_eletricidade.head()

,forecast_date,euros_per_mwh,origin_date,data_block_id
0,2021-09-01 00:00:00,92.51,2021-08-31 00:00:00,1
1,2021-09-01 01:00:00,88.90,2021-08-31 01:00:00,1
2,2021-09-01 02:00:00,87.35,2021-08-31 02:00:00,1
3,2021-09-01 03:00:00,86.88,2021-08-31 03:00:00,1
4,2021-09-01 04:00:00,88.43,2021-08-31 04:00:00,1


## Camada de Modelagem

In [4]:
# Tratamento dos dados
base_eletricidade.forecast_date = pd.to_datetime( base_eletricidade.forecast_date )

In [5]:
base_eletricidade['ano'] = base_eletricidade.forecast_date.dt.year
base_eletricidade['mes'] = base_eletricidade.forecast_date.dt.month
base_eletricidade['dia'] = base_eletricidade.forecast_date.dt.day
base_eletricidade['data'] = base_eletricidade.forecast_date.dt.date
base_eletricidade['hora'] = base_eletricidade.forecast_date.dt.hour

In [6]:
base_eletricidade.head()

,forecast_date,euros_per_mwh,origin_date,data_block_id,ano,mes,dia,data,hora
0,2021-09-01 00:00:00,92.51,2021-08-31 00:00:00,1,2021,9,1,2021-09-01,0
1,2021-09-01 01:00:00,88.90,2021-08-31 01:00:00,1,2021,9,1,2021-09-01,1
2,2021-09-01 02:00:00,87.35,2021-08-31 02:00:00,1,2021,9,1,2021-09-01,2
3,2021-09-01 03:00:00,86.88,2021-08-31 03:00:00,1,2021,9,1,2021-09-01,3
4,2021-09-01 04:00:00,88.43,2021-08-31 04:00:00,1,2021,9,1,2021-09-01,4


## Análise inicial

In [7]:
anl_media_preco = base_eletricidade.groupby( by=['data']). agg(
    media_preco = ('euros_per_mwh', 'mean')
)

anl_media_preco['mm30d'] = anl_media_preco.media_preco.rolling(window=30).mean()
anl_media_preco['mm7d'] = anl_media_preco.media_preco.rolling(window=7).mean()
#Agrupa os dados por cada valor único da coluna data e calcula a média de preço por hora

In [8]:
anl_media_preco.head()

,media_preco,mm30d,mm7d
data,,,
2021-09-01,110.467083,NaN,NaN
2021-09-02,112.631250,NaN,NaN
2021-09-03,107.429583,NaN,NaN
2021-09-04,106.112917,NaN,NaN
2021-09-05,99.045417,NaN,NaN


## Montagem do gráfico

In [9]:
# Criar figura
Figura = go.Figure()

# Adicionar série temporal diária
Figura.add_trace(
    go.Scatter(
        x=anl_media_preco.index,
        y=anl_media_preco['media_preco'],
        mode='lines',
        name='Diário',
        line=dict(color='#adadad')
    )
)

# Adicionar média móvel de 7 dias
Figura.add_trace(
    go.Scatter(
        x=anl_media_preco.index,
        y=anl_media_preco['mm7d'],
        mode='lines',
        name='mm7d',
        line=dict(color='blue', width=2)
    )
)

# Adicionar média móvel de 30 dias com janela rolante de 20 dias
Figura.add_trace(
    go.Scatter(
        x=anl_media_preco.index,
        y=anl_media_preco['mm30d'].rolling(window=20).mean(),
        mode='lines',
        name='mm30d',
        line=dict(color='#4f390b', width=3)
    )
)

# Títulos e labels
Figura.update_layout(
    title='Série Temporal | Preço megawatt-hora (MWh) €',
    xaxis_title='Data',
    yaxis_title='Preço em EURO €',
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='center',
        x=0.5,
    ),
    height=600,
    width=1200
)

Figura

## Box plot 

In [10]:
base_eletricidade['data_boxplot'] = base_eletricidade['ano'].astype('str') + '-' + base_eletricidade['mes'].astype('str')
base_eletricidade.head()

,forecast_date,euros_per_mwh,origin_date,data_block_id,ano,mes,dia,data,hora,data_boxplot
0,2021-09-01 00:00:00,92.51,2021-08-31 00:00:00,1,2021,9,1,2021-09-01,0,2021-9
1,2021-09-01 01:00:00,88.90,2021-08-31 01:00:00,1,2021,9,1,2021-09-01,1,2021-9
2,2021-09-01 02:00:00,87.35,2021-08-31 02:00:00,1,2021,9,1,2021-09-01,2,2021-9
3,2021-09-01 03:00:00,86.88,2021-08-31 03:00:00,1,2021,9,1,2021-09-01,3,2021-9
4,2021-09-01 04:00:00,88.43,2021-08-31 04:00:00,1,2021,9,1,2021-09-01,4,2021-9


In [11]:
dados_filtrados = base_eletricidade.loc[base_eletricidade['euros_per_mwh'] < 4000]

# Criar figura
Figura2 = go.Figure()

# Adicionar um boxplot para cada mês (assumindo que 'data_boxplot' contém a categoria mensal)
for categoria in dados_filtrados['data_boxplot'].unique():

    Figura2.add_trace(go.Box(
        y=dados_filtrados[dados_filtrados['data_boxplot'] == categoria]['euros_per_mwh'],
        name=categoria,
        boxmean='sd',  # Para mostrar a média e desvio padrão no boxplot
        width=0.5,
        marker_color='#636efa'
    ))

# Títulos e labels
Figura2.update_layout(
    title='Distribuição de Preço megawatt-hora (MWh) € | Mensal',
    xaxis_title='Mês',
    yaxis_title='Preço em EURO €',
    xaxis={'type': 'category'},  # Para garantir que os meses sejam categóricos
    height=500,
    width=1200,
    showlegend=False
)

# Rotacionar os rótulos do eixo x
Figura2.update_xaxes(tickangle=90)

## Mapa de calor 

In [13]:
def gerar_grafico_estudo( dados, ano, mes ):

  # Filtrar o ano e mes que o usuário está setando no FRONT-END
  Filtro = dados.loc[ (dados.ano == ano) & (dados.mes == mes) ]

  # Analise
  anl_estudo = Filtro.groupby( by=['dia', 'hora'] ).agg(
    media_preco = ('euros_per_mwh', 'mean')
  ).reset_index()

  # Pivotar a tabela
  anl_estudo = anl_estudo.pivot_table( index='hora', columns='dia', values='media_preco')

  # Ordenacao
  anl_estudo = anl_estudo.sort_index()

  # Ajuste no index
  anl_estudo.index = anl_estudo.index.astype(str)

  # Criar heatmap
  Figura3 = go.Figure(
      data=go.Heatmap(
        z=anl_estudo.values,
        x=anl_estudo.columns,
        y=anl_estudo.index,
        colorscale='Reds',
        showscale=True,
        colorbar=dict(thickness=10, len=0.25)
    )
  )

  # Títulos e labels
  Figura3.update_layout(
      title=f'Mapa de Calor {ano}, comparação dias x hora [ Janeiro]',
      xaxis_title='Dias',
      yaxis_title='Horário',
      height=700,
      width=900
  )

  return Figura3

In [14]:
gerar_grafico_estudo(base_eletricidade, 2022, 1)

In [15]:
# Analise horário vs dia
base_eletricidade.loc[ (base_eletricidade.ano == 2023) & (base_eletricidade.mes == 1) ].groupby(
    by=['dia', 'hora'] ).agg(
      media_preco = ('euros_per_mwh', 'mean')
        ).reset_index().pivot_table(
            index='hora', columns='dia', values='media_preco')

dia,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
hora,,,,,,,,,,,,,,,,,,,,,
0,2.01,57.91,78.81,82.88,30.03,107.21,71.67,24.44,69.04,76.05,...,79.40,108.55,99.60,102.22,100.81,78.56,115.92,29.76,13.09,25.36
1,1.38,51.67,73.93,78.84,28.15,101.10,69.93,52.50,64.38,79.95,...,79.17,108.53,99.53,9.00,71.34,81.91,111.72,29.32,5.76,25.20
2,0.09,52.86,73.94,69.93,27.68,96.73,69.94,46.16,60.28,78.82,...,80.00,91.16,63.10,4.58,70.06,91.56,79.36,28.37,4.95,25.58
3,0.08,44.16,71.44,69.26,27.76,82.34,70.95,34.41,39.41,80.06,...,81.43,108.50,88.29,4.10,100.80,94.96,45.01,28.23,4.05,26.30
4,0.05,50.08,72.33,71.75,28.63,81.93,73.83,44.38,47.34,83.16,...,81.94,108.55,99.59,6.08,106.52,109.42,85.07,28.62,9.03,29.24
5,0.08,70.73,77.14,81.73,39.80,98.11,75.26,9.92,73.11,104.63,...,108.59,146.71,108.29,40.04,106.56,123.19,111.75,26.99,26.74,30.36
6,0.09,105.08,102.48,112.76,62.70,100.72,81.83,49.71,112.29,117.80,...,62.25,189.96,99.60,77.99,160.39,147.39,50.99,27.26,58.21,103.99
7,0.53,138.19,154.96,126.72,85.53,128.30,75.97,57.67,156.16,149.40,...,64.92,226.54,134.59,163.99,182.49,176.10,101.50,29.18,82.01,110.67
8,2.03,145.98,163.92,127.91,93.54,135.57,104.86,75.53,175.68,169.15,...,86.01,251.73,123.29,119.95,198.95,195.00,111.72,30.90,106.73,149.56


In [16]:
# Analise horário vs dia
anl_horaDia = base_eletricidade.loc[ base_eletricidade.ano == 2023 ].groupby(
    by=['dia', 'hora'] ).agg(
      media_preco = ('euros_per_mwh', 'mean')
        ).reset_index().pivot_table(
            index='hora', columns='dia', values='media_preco')

# ajustando index
anl_horaDia = anl_horaDia.sort_index()
anl_horaDia.index = anl_horaDia.index.astype(str)

anl_horaDia.head()

dia,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
hora,,,,,,,,,,,,,,,,,,,,,
0,43.696,74.098,58.102,86.034,85.822,97.226,86.870,55.424,79.102,63.686,...,69.996,70.852,59.588,68.022,65.354,73.992,84.978,46.7075,39.0975,31.300
1,37.118,71.252,54.498,70.072,83.772,89.836,83.672,60.072,67.004,48.180,...,64.164,64.482,56.212,42.952,54.686,74.106,79.802,44.1200,35.2675,31.550
2,40.264,70.402,54.216,82.364,81.380,88.436,83.614,57.736,53.454,47.924,...,64.354,51.652,53.788,33.640,55.120,82.014,58.696,42.9425,31.0950,31.310
3,45.320,68.894,57.900,81.454,81.900,88.548,84.450,57.514,50.624,55.288,...,68.178,54.562,65.938,30.992,46.948,83.966,50.800,43.0500,29.3925,31.990
4,55.786,74.632,76.896,76.720,85.524,95.878,87.416,65.298,71.742,66.236,...,70.638,65.070,71.968,34.148,52.932,88.344,72.826,56.8075,35.9050,34.455


In [17]:
# Criar heatmap
Figura3 = go.Figure(data=go.Heatmap(
    z=anl_horaDia.values,
    x=anl_horaDia.columns,
    y=anl_horaDia.index,
    colorscale='Reds',
    showscale=True,
    colorbar=dict(thickness=10, len=0.25)  # Shrink colorbar
))

# Títulos e labels
Figura3.update_layout(
    title='Mapa de Calor 2023, comparação dias x  hora',
    xaxis_title='Dias',
    yaxis_title='Horário',
    height=700,
    width=900
)

## Dados para o dashboard 

In [18]:
anl_media_bigNumber = base_eletricidade.groupby( by=['ano'] ).agg( media= ('euros_per_mwh', 'mean') ).reset_index()

anl_media_2024 = pd.DataFrame({
    'ano' : 2024,
    'media' : base_eletricidade['euros_per_mwh'].mean()
}, index=[0])

anl_media_bigNumber = pd.concat( [anl_media_bigNumber, anl_media_2024] )

anl_media_bigNumber

,ano,media
0,2021,137.159225
1,2022,192.834105
2,2023,86.203626
0,2024,157.064176


In [21]:
! pip install pyarrow

Defaulting to user installation because normal site-packages is not writeable


In [24]:
anl_media_bigNumber.to_parquet('dados_bignumber.parquet')

In [25]:
anl_media_preco.to_parquet('dados_serietemporal.parquet')

In [26]:
dados_filtrados[['euros_per_mwh', 'data_boxplot']].to_parquet('dados_boxplot.parquet')

In [27]:
base_eletricidade[['ano', 'mes', 'dia', 'hora', 'euros_per_mwh']].to_parquet('dados_estudo.parquet')